In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
DYNAMO_DEBUG = False
COMPILE = True
SHOW_ANIMATION = False

In [ ]:
%env TORCH_LOGS=dynamo 
%env TORCHDYNAMO_VERBOSE=1

In [ ]:
import torch
import matplotlib.pyplot as plt
from flipper_training.configs import (
    WorldConfig,
    RobotModelConfig,
    PhysicsEngineConfig,
)
from flipper_training.engine.engine import DPhysicsEngine, PhysicsState
from flipper_training.utils.dynamics import *
from flipper_training.utils.geometry import *
from flipper_training.utils.environment import *
from flipper_training.utils.numerical import *
from collections import deque

In [ ]:
from flipper_training.utils.torch_utils import set_device

device = "cuda:0"
device = set_device(device)

In [ ]:
from flipper_training.vis.static_vis import *

In [ ]:
torch.random.manual_seed(420)

In [ ]:
num_robots = 1

In [ ]:
# Heightmap setup - use torch's XY indexing !!!!!
grid_res = 0.05  # 5cm per grid cell
max_coord = 3.2  # meters
DIM = int(2 * max_coord / grid_res)
xint = torch.linspace(-max_coord, max_coord, DIM)
yint = torch.linspace(-max_coord, max_coord, DIM)
x, y = torch.meshgrid(xint, yint, indexing="xy")

In [ ]:
# # gaussian hm
# z = (
#     (
#         1.0 * torch.exp(-0.5 * ((x - 0) ** 2 + (y - 4) ** 2))
#         + 5.0 * torch.exp(-0.3 * ((x - 1) ** 2 + (y + 2) ** 2))
#         + 2.0 * torch.exp(-0.1 * ((x + max_coord) ** 2 + (y + max_coord) ** 2))
#     )
#     + 0.01 * torch.randn_like(x)
#     + torch.exp(-0.03 * ((x + 5) ** 2 + (y + 5) ** 2))
# )
# block hm
z = torch.zeros_like(x)
for thresh in [1.0, 0, -1.0, -2]:
    z[torch.logical_and(x > -thresh, y < thresh)] += 0.2

# quadratic hill hm
# z = 1 - 0.01 * ((x-1)**8 + (y+3)**8) + 0.01 * torch.randn_like(x)
# z.clamp_(min=0)
# zero hm
# z = torch.zeros_like(x)
x_grid = x.repeat(num_robots, 1, 1)
y_grid = y.repeat(num_robots, 1, 1)
z_grid = z.repeat(num_robots, 1, 1)
x_grid.shape

In [ ]:
from flipper_training.heightmaps.multi_gaussian import *

# Heightmap setup - use torch's XY indexing !!!!!
grid_res = 0.05  # 5cm per grid cell
max_coord = 3.2  # meters
heighmap_gen = MultiGaussianHeightmapGenerator(
    min_gaussians=400,
    max_gaussians=600,
    min_height_fraction=0.03,
    max_height_fraction=0.18,
    min_std_fraction=0.03,
    max_std_fraction=0.08,
    min_sigma_ratio=0.6,
)
x_grid, y_grid, z_grid, extras = heighmap_gen(grid_res, max_coord, num_robots)

In [ ]:
plot_grids_xyz(x_grid[0], y_grid[0], z_grid[0])

In [ ]:
# Instatiate the physics config
robot_model = RobotModelConfig(kind="marv", points_per_driving_part=384, points_per_body=512)
terrain_config = WorldConfig(
    x_grid=x_grid,
    y_grid=y_grid,
    z_grid=z_grid,
    grid_res=grid_res,
    max_coord=max_coord,
    k_stiffness=40000,
    k_friction_lat=0.5,
    k_friction_lon=0.8,
)
physics_config = PhysicsEngineConfig(num_robots=num_robots, damping_alpha=5.0, dt=0.01)

In [ ]:
# Controls
traj_length = 8.0  # seconds
n_iters = int(traj_length / physics_config.dt)
speed = 1.0  # m/s forward
omega = 0.0  # rad/s yaw
controls = robot_model.vw_to_vels(speed, omega)
flipper_controls = torch.zeros_like(controls)

In [ ]:
for cfg in [robot_model, terrain_config, physics_config]:
    cfg.to(device)

In [ ]:
engine = DPhysicsEngine(physics_config, robot_model, device)

In [ ]:
from flipper_training.utils.geometry import euler_to_quaternion


x0 = torch.tensor([[-2.5, -2.5, 0.400]]).to(device).repeat(num_robots, 1)
xd0 = torch.zeros_like(x0)
q0 = euler_to_quaternion(*torch.tensor([0, 0, 0.0 * torch.pi])).to(device).repeat(num_robots, 1)
omega0 = torch.zeros_like(x0)
thetas0 = torch.zeros(num_robots, robot_model.num_driving_parts).to(device)
controls_all = torch.cat((controls, flipper_controls), dim=-1).repeat(n_iters, num_robots, 1).to(device)

In [ ]:
# Set joint rotational velocities, we want to follow a sine wave, so we set the joint velocities to the derivative of the sine wave
# We want to go +- pi/6 5 times in 10 seconds
amplitude = 0 * torch.pi / 4
periods = traj_length / 10.0
rot_vels = torch.cos(torch.linspace(0, periods * 2 * np.pi, n_iters)) * amplitude
rot_vels = rot_vels.unsqueeze(-1).repeat(1, num_robots)
controls_all[:, :, robot_model.num_driving_parts] = rot_vels
controls_all[:, :, robot_model.num_driving_parts + 1] = rot_vels
controls_all[:, :, robot_model.num_driving_parts + 2] = -rot_vels
controls_all[:, :, robot_model.num_driving_parts + 3] = -rot_vels

In [ ]:
# Set the flippers to an fixed position at the beginning
angles_deg = torch.tensor([-90.0, -90.0, 90.0, 90.0]).to(device)
angles = torch.deg2rad(angles_deg)
max_joint_vel = robot_model.joint_max_pivot_vels
min_joint_vel = -robot_model.joint_max_pivot_vels
times = (angles >= 0).float() * (angles / max_joint_vel) + (angles < 0).float() * (angles / min_joint_vel)
steps = (times / physics_config.dt).int()
vels = angles / times
vels[torch.isnan(vels)] = 0.0
# controls_all[: steps[0], :, robot_model.num_driving_parts] = vels[0]
# controls_all[: steps[1], :, robot_model.num_driving_parts + 1] = vels[1]
# controls_all[: steps[2], :, robot_model.num_driving_parts + 2] = vels[2]
# controls_all[: steps[3], :, robot_model.num_driving_parts + 3] = vels[3]

In [ ]:
init_state = PhysicsState(x0, xd0, q0, omega0, thetas0)

In [ ]:
bench_state = init_state.new_zeros()

In [ ]:
compile_opts = {"max-autotune": True, "triton.cudagraphs": True, "coordinate_descent_tuning": True}
if DYNAMO_DEBUG:
    compile_opts |= {"trace.enabled": True, "trace.graph_diagram": True}

In [ ]:
print(torch._inductor.list_mode_options())
print(torch._inductor.list_options())

In [ ]:
if COMPILE:
    # run all torch compilation
    engine = torch.compile(engine, options=compile_opts)
    _ = engine(bench_state, controls_all[0], terrain_config)

In [ ]:
states = deque(maxlen=n_iters)
dstates = deque(maxlen=n_iters)
auxs = deque(maxlen=n_iters)

In [ ]:
from flipper_training.engine.engine_state import (
    vectorize_iter_of_states as vectorize_states,
)

In [ ]:
%%time 
state = init_state
for i in range(n_iters):
    state, der, aux = engine(state, controls_all[i], terrain_config)
    if compile:
        state = state.clone()
        der = der.clone()
        aux = aux.clone()
    states.append(state)
    dstates.append(der)
    auxs.append(aux)

In [ ]:
states_vec = vectorize_states(states)
dstates_vec = vectorize_states(dstates)
aux_vec = vectorize_states(auxs)

In [ ]:
plot_birdview_trajectory(terrain_config, states, iter_step=500)

In [ ]:
ROBOT_IDX = 0

In [ ]:
# Plot components of acceleration
ay, ax, az = dstates_vec.xdd[:, ROBOT_IDX].T.cpu().numpy()
plt.figure(figsize=(10, 5))
plt.title("Acceleration")
plt.plot(ay, label="ay")
plt.plot(ax, label="ax")
plt.plot(az, label="az")
plt.legend()
plt.show()

# Plot components of torque
t1, t2, t3 = aux_vec.torque[:, ROBOT_IDX].T.cpu().numpy()
plt.figure(figsize=(10, 5))
plt.title("Torque")
plt.plot(t1, label="t1")
plt.plot(t2, label="t2")
plt.plot(t3, label="t3")
plt.grid()
plt.xlabel("Time step")
plt.ylabel("Torque (Nm) at CoG")
plt.legend()
plt.show()

# Plot components of velocity
v = dstates_vec.xd[:, ROBOT_IDX].cpu().numpy()
plt.figure(figsize=(10, 5))
plt.title("Velocity")
plt.plot((v**2).sum(-1) ** 0.5, label="v")
plt.plot(v[:, 0], label="vx")
plt.plot(v[:, 1], label="vy")
plt.plot(v[:, 2], label="vz")
plt.yticks([-1, 0, 1])
plt.legend()
plt.grid()
plt.show()

# Plot components of rotational velocity
w = states_vec.omega[:, ROBOT_IDX].cpu().numpy()
plt.figure(figsize=(10, 5))
plt.title("Rotational velocity")
plt.plot((w**2).sum(-1) ** 0.5, label="w")
plt.plot(w[:, 0], label="wx")
plt.plot(w[:, 1], label="wy")
plt.plot(w[:, 2], label="wz")
plt.yticks([-1, 0, 1])
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Height of the robot
plt.figure(figsize=(10, 5), dpi=200)
z = states_vec.x[:, ROBOT_IDX, 2].cpu().numpy()
plt.plot(z)

In [ ]:
plot_3d_trajectory(terrain_config, states, auxs)

In [ ]:
if SHOW_ANIMATION:
    from flipper_training.vis.animator import animate_trajectory

    ani = animate_trajectory(
        terrain_config,
        physics_config,
        states,
        auxs,
        ROBOT_IDX,
        show_xd_points=False,
        show_cog_coords=False,
        show_friction_forces=False,
        show_spring_forces=False,
        show_act_forces=True,
    )